In [ ]:
#%pip install dotenv

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# atur timezone
#%pip install pytz

Note: you may need to restart the kernel to use updated packages.


In [ ]:
# untuk melakukan geocoding
#%pip install geopy

^C
Note: you may need to restart the kernel to use updated packages.


In [5]:
# Standart Library imports
import pandas as pd
import numpy as np
import pandas_gbq as gbq
import requests

# DATETIME AND TIMEZONE LIBRARY 
from datetime import datetime, timezone
from pytz import timezone as tz

# GEOCODING LIBRARY
from geopy.geocoders import Nominatim

# GOOGLE API LIBRARY
from google.oauth2 import service_account

# Logging
import logging
# set up logger
logging.basicConfig(level=logging.INFO)

Table of Content
1. Extract
2. Transform
3. Load

# 1. DATA EXTRACTION from USGS Public API

In [6]:
# define base URL

# base url for realtime earthquake data (past 24 hours)
BASE_URL = "https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_day.geojson"

# base url for historical earthquake data
BASE_URL_HISTORICAL = "https://earthquake.usgs.gov/fdsnws/event/1/query"

In [7]:
# Get All realtime earthquake data
response = requests.get(BASE_URL)
data = response.json()
data

{'type': 'FeatureCollection',
 'metadata': {'generated': 1752116724000,
  'url': 'https://earthquake.usgs.gov/earthquakes/feed/v1.0/summary/all_day.geojson',
  'title': 'USGS All Earthquakes, Past Day',
  'status': 200,
  'api': '1.14.1',
  'count': 304},
 'features': [{'type': 'Feature',
   'properties': {'mag': 0.76,
    'place': '7 km NNW of The Geysers, CA',
    'time': 1752116372730,
    'updated': 1752116468923,
    'tz': None,
    'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/nc75207762',
    'detail': 'https://earthquake.usgs.gov/earthquakes/feed/v1.0/detail/nc75207762.geojson',
    'felt': None,
    'cdi': None,
    'mmi': None,
    'alert': None,
    'status': 'automatic',
    'tsunami': 0,
    'sig': 9,
    'net': 'nc',
    'code': '75207762',
    'ids': ',nc75207762,',
    'sources': ',nc,',
    'types': ',nearby-cities,origin,phase-data,',
    'nst': 16,
    'dmin': 0.003194,
    'rms': 0.01,
    'gap': 96,
    'magType': 'md',
    'type': 'earthquake',
    'ti

In [8]:
data['features']

[{'type': 'Feature',
  'properties': {'mag': 0.76,
   'place': '7 km NNW of The Geysers, CA',
   'time': 1752116372730,
   'updated': 1752116468923,
   'tz': None,
   'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/nc75207762',
   'detail': 'https://earthquake.usgs.gov/earthquakes/feed/v1.0/detail/nc75207762.geojson',
   'felt': None,
   'cdi': None,
   'mmi': None,
   'alert': None,
   'status': 'automatic',
   'tsunami': 0,
   'sig': 9,
   'net': 'nc',
   'code': '75207762',
   'ids': ',nc75207762,',
   'sources': ',nc,',
   'types': ',nearby-cities,origin,phase-data,',
   'nst': 16,
   'dmin': 0.003194,
   'rms': 0.01,
   'gap': 96,
   'magType': 'md',
   'type': 'earthquake',
   'title': 'M 0.8 - 7 km NNW of The Geysers, CA'},
  'geometry': {'type': 'Point',
   'coordinates': [-122.805999755859, 38.8236656188965, 2.69000005722046]},
  'id': 'nc75207762'},
 {'type': 'Feature',
  'properties': {'mag': 1.2,
   'place': '74 km NNW of Karluk, Alaska',
   'time': 1752116182034,

In [9]:
data['features'][0]['properties']

{'mag': 0.76,
 'place': '7 km NNW of The Geysers, CA',
 'time': 1752116372730,
 'updated': 1752116468923,
 'tz': None,
 'url': 'https://earthquake.usgs.gov/earthquakes/eventpage/nc75207762',
 'detail': 'https://earthquake.usgs.gov/earthquakes/feed/v1.0/detail/nc75207762.geojson',
 'felt': None,
 'cdi': None,
 'mmi': None,
 'alert': None,
 'status': 'automatic',
 'tsunami': 0,
 'sig': 9,
 'net': 'nc',
 'code': '75207762',
 'ids': ',nc75207762,',
 'sources': ',nc,',
 'types': ',nearby-cities,origin,phase-data,',
 'nst': 16,
 'dmin': 0.003194,
 'rms': 0.01,
 'gap': 96,
 'magType': 'md',
 'type': 'earthquake',
 'title': 'M 0.8 - 7 km NNW of The Geysers, CA'}

In [ ]:
# convert json to dataFrame

records = []

for feature in data['features']:
    props = feature['properties']
    coords = feature['geometry']['coordinates']  # [lon, lat, depth]

    record = {
        "id": feature.get("id"),
        "place": props.get("place"),
        "mag": props.get("mag"),
        "time": pd.to_datetime(props.get("time"), unit='ms'),
        "updated": pd.to_datetime(props.get("updated"), unit='ms'),
        "tz": props.get("tz"),
        "felt": props.get("felt"),
        "cdi": props.get("cdi"),
        "mmi": props.get("mmi"),
        "alert": props.get("alert"),
        "status": props.get("status"),
        "tsunami": props.get("tsunami"),
        "sig": props.get("sig"),
        "net": props.get("net"),
        "code": props.get("code"),
        "ids": props.get("ids"),
        "sources": props.get("sources"),
        "types": props.get("types"),
        "longitude": coords[0],
        "latitude": coords[1],
        "depth": coords[2],
        "fetched_at": datetime.now(timezone.utc)
    }
    records.append(record)

records_df = pd.DataFrame(records)

In [ ]:
records_df

,id,place,mag,time,updated,tz,felt,cdi,mmi,alert,...,sig,net,code,ids,sources,types,longitude,latitude,depth,fetched_at
0,uw62136047,"22 km ENE of Ashford, Washington",0.36,2025-07-09 03:54:32.100,2025-07-09 03:56:07.120,None,NaN,NaN,NaN,None,...,2,uw,62136047,",uw62136047,",",uw,",",origin,phase-data,",-121.763168,46.849167,-2.110000,2025-07-09 04:03:18.618657+00:00
1,ak0258qa7usu,"93 km NW of Aleneva, Alaska",2.60,2025-07-09 03:54:19.346,2025-07-09 03:55:48.029,None,NaN,NaN,NaN,None,...,104,ak,0258qa7usu,",ak0258qa7usu,",",ak,",",origin,phase-data,",-154.174500,58.575800,13.600000,2025-07-09 04:03:18.619254+00:00
2,tx2025njfuex,"47 km NW of Toyah, Texas",1.40,2025-07-09 03:47:27.056,2025-07-09 03:50:10.979,None,NaN,NaN,NaN,None,...,30,tx,2025njfuex,",tx2025njfuex,",",tx,",",origin,phase-data,",-104.126000,31.635000,5.000000,2025-07-09 04:03:18.619797+00:00
3,ok2025nhpl,"5 km E of Meridian, Oklahoma",1.80,2025-07-09 03:43:04.389,2025-07-09 03:45:58.031,None,NaN,NaN,NaN,None,...,50,ok,2025nhpl,",ok2025nhpl,",",ok,",",origin,phase-data,",-97.189034,35.846992,7.716245,2025-07-09 04:03:18.620354+00:00
4,us7000qbi2,"39 km E of Nemuro, Japan",4.60,2025-07-09 03:39:50.569,2025-07-09 03:56:11.040,None,NaN,NaN,NaN,None,...,326,us,7000qbi2,",us7000qbi2,",",us,",",origin,phase-data,",146.058900,43.347900,119.917000,2025-07-09 04:03:18.620900+00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
456,ci41016631,"0 km E of Loma Linda, CA",0.52,2025-07-08 04:31:21.830,2025-07-08 17:49:28.666,None,NaN,NaN,NaN,None,...,4,ci,41016631,",ci41016631,",",ci,",",nearby-cities,origin,phase-data,scitech-link,",-117.257500,34.048833,15.920000,2025-07-09 04:03:19.287674+00:00
457,av93046044,"57 km WNW of Tyonek, Alaska",-0.90,2025-07-08 04:28:50.720,2025-07-08 21:45:04.050,None,NaN,NaN,NaN,None,...,0,av,93046044,",av93046044,",",av,",",origin,phase-data,",-152.142167,61.249500,-3.140000,2025-07-09 04:03:19.288047+00:00
458,tx2025nhlkqe,"23 km S of Westbrook, Texas",0.90,2025-07-08 04:28:23.863,2025-07-08 23:05:34.736,None,NaN,NaN,NaN,None,...,12,tx,2025nhlkqe,",tx2025nhlkqe,",",tx,",",origin,phase-data,",-101.046000,32.148000,8.642600,2025-07-09 04:03:19.288389+00:00
459,nc75206902,"4 km NNW of The Geysers, CA",1.03,2025-07-08 04:11:39.870,2025-07-08 04:13:13.724,None,NaN,NaN,NaN,None,...,16,nc,75206902,",nc75206902,",",nc,",",nearby-cities,origin,phase-data,",-122.783997,38.803833,3.180000,2025-07-09 04:03:19.288741+00:00


# 2. Transformation

- Basic Cleaning
- Data Enrichment

In [ ]:
df = records_df.copy()

In [ ]:
# drop duplicate records
df = df.drop_duplicates(subset=['id'])

# drop rows with null magnitude or location
df = df.dropna(subset=['mag', 'place', 'latitude', 'longitude', 'time'])

In [ ]:
# Convert time columns to datetime if not already
# Convert UTC time columns to datetime with Asia/Jakarta timezone
WIB = tz('Asia/Jakarta')
df['time'] = pd.to_datetime(df['time'], errors='coerce').dt.tz_localize('UTC').dt.tz_convert(WIB)
df['updated'] = pd.to_datetime(df['updated'], errors='coerce').dt.tz_localize('UTC').dt.tz_convert(WIB)
df['fetched_at'] = pd.to_datetime(df['fetched_at'], errors='coerce').dt.tz_convert('UTC').dt.tz_convert(WIB)

In [ ]:
#  Filter only valid magnitudes
df = df[df['mag'] >= 0]

# Select relevant columns
selected_columns = [
    "id", "place", "mag", "time", "updated",
    "latitude", "longitude", "depth",
    "tsunami", "sig", "status", "alert", "types",
    "felt", "cdi", "mmi", "fetched_at"
]
df = df[selected_columns]

In [ ]:
# 2b. Data Enrichment

# Add Magnitude Category
def classify_magnitude(mag):
        if mag < 2.0:
            return "Micro"
        elif mag < 4.0:
            return "Minor"
        elif mag < 5.5:
            return "Light"
        elif mag < 7.0:
            return "Moderate"
        elif mag < 8.0:
            return "Strong"
        else:
            return "Major"

df['mag_category'] = df['mag'].apply(classify_magnitude)

In [ ]:
df.head()

,id,place,mag,time,updated,latitude,longitude,depth,tsunami,sig,status,alert,types,felt,cdi,mmi,fetched_at,mag_category
0,uw62136047,"22 km ENE of Ashford, Washington",0.36,2025-07-09 10:54:32.100000+07:00,2025-07-09 10:56:07.120000+07:00,46.849167,-121.763168,-2.110000,0,2,automatic,None,",origin,phase-data,",NaN,NaN,NaN,2025-07-09 11:03:18.618657+07:00,Micro
1,ak0258qa7usu,"93 km NW of Aleneva, Alaska",2.60,2025-07-09 10:54:19.346000+07:00,2025-07-09 10:55:48.029000+07:00,58.575800,-154.174500,13.600000,0,104,automatic,None,",origin,phase-data,",NaN,NaN,NaN,2025-07-09 11:03:18.619254+07:00,Minor
2,tx2025njfuex,"47 km NW of Toyah, Texas",1.40,2025-07-09 10:47:27.056000+07:00,2025-07-09 10:50:10.979000+07:00,31.635000,-104.126000,5.000000,0,30,automatic,None,",origin,phase-data,",NaN,NaN,NaN,2025-07-09 11:03:18.619797+07:00,Micro
3,ok2025nhpl,"5 km E of Meridian, Oklahoma",1.80,2025-07-09 10:43:04.389000+07:00,2025-07-09 10:45:58.031000+07:00,35.846992,-97.189034,7.716245,0,50,automatic,None,",origin,phase-data,",NaN,NaN,NaN,2025-07-09 11:03:18.620354+07:00,Micro
4,us7000qbi2,"39 km E of Nemuro, Japan",4.60,2025-07-09 10:39:50.569000+07:00,2025-07-09 10:56:11.040000+07:00,43.347900,146.058900,119.917000,0,326,reviewed,None,",origin,phase-data,",NaN,NaN,NaN,2025-07-09 11:03:18.620900+07:00,Light


In [ ]:
# Add address details
def get_address_detail(latitude, longitude):
    result = {}
    coder = Nominatim(user_agent="myGeocoder", timeout=4)
    
    # print(coder.reverse(f"{latitude}, {longitude}"))
    address = coder.reverse(f"{latitude}, {longitude}")

    result["city"] = address.raw.get("address").get("city") if address else np.nan
    result["state"] = address.raw.get("address").get("state") if address else np.nan
    result["country"] = address.raw.get("address").get("country") if address else np.nan
    return result

address_detail = df.apply(lambda row: get_address_detail(row['latitude'], row['longitude']), axis=1, result_type='expand')
df = pd.concat([df, address_detail], axis=1)

In [ ]:
# Add time-based columns
df['day_of_week'] = df['time'].dt.day_name()
df['hour_of_day'] = df['time'].dt.hour

In [ ]:
# 2c. Save data into csv
df.to_csv(f"../data/earthquakes-{datetime.now().strftime('%Y%m%d')}.csv", index=False)

In [ ]:
# BigQuery configuration
PROJECT_ID = "earthquake-jcdsdevi"
TABLE_ID = "earthquakes.events" # dataset.id
CREDENTIALS_PATH = "../config/earthquake-jcdsdevi-7e3c550888de.json"

In [ ]:
credentials = service_account.Credentials.from_service_account_file(CREDENTIALS_PATH)

In [ ]:
gbq.to_gbq(
    dataframe=df,
    destination_table=TABLE_ID,
    project_id=PROJECT_ID,
    credentials=credentials,
    if_exists="replace",
)